[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/ee-LandsatLinkr/landsatlinkr_run_template.ipynb)

# Environment setup



Import the Earth Engine API and authenticate to the service

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

Get LandsatLinkr library.

In [ ]:
!rm -r /content/ee-LandsatLinkr
!git clone https://github.com/gee-community/ee-LandsatLinkr --quiet
import sys
sys.path.append("/content/ee-LandsatLinkr")
from landsatlinkr import *

# Identity MSS WRS-1 tile ID(s)


1. Visit: https://code.earthengine.google.com/978fb6b2d523007918d77f2b35048c4a 
2. In the map, zoom to your study area and select its location to reveal the WRS-1 tile IDs at that location (this may take a minute or two to load)
3. Copy the WRS-1 tile ID from the pop-up window. If your study area overlaps with two or more tile footprints, you should note all of the WRS-1 tile IDs and process as many tiles that intersect the study region one at a time – the results can be composited later. For now, select one WRS-1 tile ID to begin with (you can only process one tile ID at a time).
4. Paste this WRS-1 tile ID into your run script as the WRS_1_GRANULE variable:

In [ ]:
PROJECT_DIR = 'projects/ee-braaten/assets/llr' 
WRS_1_GRANULE =  '050028' #(saint helens) # '041029' (west yellowstone) #    '049030' (western oregon)

Create a project directory for this MSS WRS-1 ID

TODO: does not warn you if it failed

In [ ]:
createProjectDir(PROJECT_DIR, WRS_1_GRANULE)

Set variables TODO: should allow people to select the end year, right now I think it is hard-coded to 2021

In [ ]:
CRS = 'EPSG:3857'
DOY_RANGE = [160, 254]
MAX_CLOUD = 50
MAX_GEOM_RMSE = 0.5

LT_PARAMS = {
    'ftvBands': ['tcb', 'tcg', 'tcw'], #['red', 'green', 'blue'],  # , 'tcb', 'tcg', 'tcw'
    'maxSegments': 10,
    'spikeThreshold': 0.7,
    'vertexCountOvershoot': None,
    'preventOneYearRecovery': None,
    'recoveryThreshold': 0.5,
    'pvalThreshold': None,
    'bestModelProportion': None,
    'minObservationsNeeded': None,
    'scale': 30
}

params = {
  'maxRmseVerify': MAX_GEOM_RMSE,
  'maxCloudCover': MAX_CLOUD,
  'doyRange': DOY_RANGE,
  'wrs1': WRS_1_GRANULE,
  'crs': CRS,
  'excludeIds': [],
  'baseDir': PROJECT_DIR + '/' + WRS_1_GRANULE,
  'projectDir': PROJECT_DIR,
  'ltParams': LT_PARAMS
}

Preview MSS WRS-1 images, note bad images and add them to the `EXCLUDE_IDS` list. Run until all bad images are removed.

In [ ]:
viewWrs1Col(params)

In [ ]:
# EXCLUDE_IDS = [
#   'LM10410291974209GDS03',
#   'LM20410291975231GDS02',
#   'LM20410291975249AAA04',
#   'LM20410291976172GDS04',
#   'LM10410291976181AAA02',
#   'LM20410291976190GDS02',
#   'LM10410291976253AAA05',
#   'LM20410291977220GDS04',
#   'LM20410291978179AAA02',
#   'LM30410291978188AAA02',
#   'LM20410291978197AAA02',
#   'LM20410291978215AAA02',
#   'LM20410291978233AAA02',
#   'LM30410291978242AAA02',
#   'LM20410291978251AAA02',
#   'LM20410291979174XXX01',
#   'LM20410291979192AAA05',
#   'LM30410291979201AAA08',
#   'LM20410291980169AAA08',
#   'LM20410291980241AAA05',
#   'LM20410291981199AAA03'
# ]

EXCLUDE_IDS = ['LM30500281982248AAA03']
params['excludeIds'] = EXCLUDE_IDS

Make an MSS reference image. Took about 15-25 minutes

In [ ]:
mssRefTask = exportMssRefImg(params)
monitorTaskStatus(mssRefTask)

Prepare MSS images TODO: print a message about what this step is doing, similar to above

In [ ]:
mssWrs1ToWrs2Task = processMssWrs1Imgs(params)
monitorTaskStatus(mssWrs1ToWrs2Task)

Develop a relationship between MSS and TM images.

In [ ]:
mss2TmInfoTask = exportMss2TmCoefCol(params)
monitorTaskStatus(mss2TmInfoTask)

Make MSS imagery match TM.

In [ ]:
mss2TmTask = exportFinalCorrectedMssCol(params)
monitorTaskStatus(mss2TmTask)

Run LandTrendr.



In [ ]:
ltTask = exportLt(params)
monitorTaskStatus(ltTask)